In [162]:
import requests
from bs4 import BeautifulSoup

r = requests.get('https://www.bbcgoodfood.com/recipes/hummus-without-tahini')

In [163]:
def remove_tags(html):
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
 
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [164]:
len(remove_tags(r.content))

9299

In [165]:
from vertexai.preview.language_models import TextGenerationModel

model = TextGenerationModel.from_pretrained("text-bison@001")

In [166]:
parameters = {
    "temperature": .2,
    "max_output_tokens": 1024,
    "top_p": .8,
    "top_k": 40
}

In [179]:
text = f"""
    You have the following text:
    {remove_tags(r.content)}
    
    Please write me a json file that respects the following structure:
    class Recipe(BaseModel):
        id: str
        name: str
        image_url: str
        category: str
        description: str
        steps: List[str]
        ingredients: List[Ingredient]
        reviews: Optional[int]
        rating: Optional[int]
        
    The ingredient is of the form:
    class Ingredient(BaseModel):
        name: str
        quantity: str
        measure_unit: str
"""

In [180]:
len(text)

9814

In [181]:
response = model.predict(
    text, **parameters,
)

In [182]:
print(response.text)

```json
{
  "id": "1234567890",
  "name": "Hummus without tahini",
  "image_url": "https://i.imgur.com/image.jpg",
  "category": "Lunch",
  "description": "Enjoy our easy, budget-friendly hummus recipe with your favourite sides, such as pittas, crackers and crudités. Add more liquid from the can of chickpeas for a smoother hummus, if you like. Dairy-free Egg-free Gluten-free Vegan",
  "steps": ["Tip the chickpeas, garlic, lemon juice and olive oil into a small blender or food processor. Season and add 1 tbsp of the reserved liquid from the can of chickpeas.", "Blitz together until smooth, adding more of the reserved chickpea liquid to make a smoother hummus, if you like. Tip the hummus into a bowl, drizzle with more olive oil and chill until ready to serve. Serve with pittas or vegetable crudités."],
  "ingredients": [
    {
      "name": "400g can chickpeas, drained (liquid reserved)",
      "quantity": "400g",
      "measure_unit": "can"
    },
    {
      "name": "½ small garlic clo

In [184]:
import json

with open("recipe.json", "w") as fp:
    json.dump(json.loads(response.text.replace("```json", "").replace("```", "")), fp, indent=4, ensure_ascii=False)